In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
from openai import AzureOpenAI
import jwt

github_token = os.getenv("GITHUB_TOKEN")
endpoint = https://models.inference.ai.azure.com/

headers = {
    "Authorization": f"Bearer {github_token}",
    "Accept": "application/vnd.github.v3+json",
}

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=github_token,
    api_version="2025-01-01-preview",
)

# instruction_prompt = "Read the uncommitted changes in the workspace and understand what has been changed, then read all the instructions files under the `.github/instructions` folder and see if the changes you made invalidate any of the instructions. If so, provide a proposal to users on how to update the instructions to reflect the changes you made. If no updates are needed, just say \"No updates needed\". Be concise and conservative with your suggestions. Only suggest changes that are absolutely necessary."

response = client.chat.completions.create(
	messages=[
		# {"role": "system", "content": system_prompt},
		# {"role": "user", "content": instruction_prompt},
		{"role": "user", "content": "hello"}
	],
	model="gpt-4.1",
	temperature=0,
	max_tokens=10,
	top_p=1,
)
print(response.choices[0].message.content)